In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
dir_root = ''
# If using google colab
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive/')
    dir_root = '/content/drive/MyDrive/Colab Notebooks/ESE546/hw2'

print(dir_root)


Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/ESE546/hw2


### (a) Plot the training and validation losses and errors as a function of the number of epochs


 The model currently does not achieve less than 12% validation error, you have to tweak the parameters to get it.

In [3]:
# Define the dataset directory
data_dir = os.path.join(dir_root, 'data')
print(data_dir)

if not os.path.exists(os.path.join(data_dir, 'cifar-10-batches-py')):
    download = True
    print('Dataset not found, downloading...')
else:
    download = False
    print('Dataset found, not downloading.')

# Reading in the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True,
                                        download=download, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root=data_dir, train=False,
                                       download=download, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# Defining the model
class View(nn.Module):
    def __init__(self,o):
        super().__init__()
        self.o = o

    def forward(self,x):
        return x.view(-1, self.o)

class allcnn_t(nn.Module):
    def __init__(self, c1=96, c2= 192):
        super().__init__()
        d = 0.5

        def convbn(ci,co,ksz,s=1,pz=0):
            return nn.Sequential(
                nn.Conv2d(ci,co,ksz,stride=s,padding=pz),
                nn.ReLU(True),
                nn.BatchNorm2d(co))

        self.m = nn.Sequential(
            nn.Dropout(0.2),
            convbn(3,c1,3,1,1),
            convbn(c1,c1,3,1,1),
            convbn(c1,c1,3,2,1),
            nn.Dropout(d),
            convbn(c1,c2,3,1,1),
            convbn(c2,c2,3,1,1),
            convbn(c2,c2,3,2,1),
            nn.Dropout(d),
            convbn(c2,c2,3,1,1),
            convbn(c2,c2,3,1,1),
            convbn(c2,10,1,1),
            nn.AvgPool2d(8),
            View(10))

        print('Num parameters: ', sum([p.numel() for p in self.m.parameters()]))

    def forward(self, x):
        return self.m(x)

/content/drive/MyDrive/Colab Notebooks/ESE546/hw2/data
Dataset found, not downloading.


In [4]:
from torch.utils.tensorboard import SummaryWriter

# Initialize the logger
logger = SummaryWriter(os.path.join(dir_root, 'runs/cnn_experiment'))

In [5]:
# # The training loop for 50 epochs

# def train(net, optimizer, criterion, train_loader, test_loader, epochs, model_name, plot):
#     model = net.to(device)
#     total_step = len(train_loader)
#     overall_step = 0
#     train_loss_values = []
#     train_error = []
#     val_loss_values = []
#     val_error = []
#     for epoch in range(epochs):
#         correct = 0
#         total = 0
#         flag = 0
#         running_loss = 0.0
#         if epoch == 25 and flag == 0:
#           for op_params in optimizer.param_groups:
#             op_params['lr'] = 0.001
#           flag = 1
#         for i, (images, labels) in enumerate(train_loader):
#             # Move tensors to configured device
#             images = images.to(device)
#             labels = labels.to(device)
#             #Forward Pass
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             optimizer.zero_grad()
#             loss.backward()
#             running_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#             optimizer.step()
#             if (i+1) % 1000 == 0:
#               print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))
#             if plot:
#               info = { ('loss_' + model_name): loss.item() }

#               for tag, value in info.items():
#                 logger.add_scalar(tag, value, overall_step+1)
#         train_loss_values.append(running_loss)
#         train_error.append(100-100*correct/total)

#         model.eval()
#         with torch.no_grad():
#             correct = 0
#             total = 0
#             for i, (images, labels) in enumerate(test_loader):
#                 images = images.to(device)
#                 labels = labels.to(device)
#                 outputs = model(images)
#                 _, predicted = torch.max(outputs.data, 1)
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()

#         print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))
#         val_error.append(100-100*correct/total)
#         val_loss_values.append(running_loss)
#     return val_error,val_loss_values,train_error,train_loss_values


In [6]:
# model_path_50 = os.path.join(dir_root, 'runs/hw2p3_model_1_50_epoch.pt')

# if not os.path.exists(model_path_50):
#     model = allcnn_t().to(device)
#     epochs = 50
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001, nesterov=True)
#     val_error, val_loss_values, train_error, train_loss_values = train(model, optimizer, criterion, trainloader, testloader, epochs, 'cnn_curve', True)
#     torch.save(model, model_path_50)
# else:
#     print(f"Model already exists at {model_path_50}, skipping training.")

In [7]:
# Training 100 epochs using the same configuration, except that:
# Learning rate of 0.1 for the first 40 epochs, then 0.01 for the next 40 epochs
# and then 0.001 for the final 20 epochs.
# Weight decay = 1e-3
# SGD with Nesterov’s momentum of 0.9
# Plot the training and validation losses and errors with the number of epochs


def adjust_learning_rate(optimizer, epoch, lr_array):
    if epoch < 40:
        lr = lr_array[0]
    elif epoch < 80:
        lr = lr_array[1]
    else:
        lr = lr_array[2]
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def train(net, optimizer, criterion, train_loader, test_loader, epochs, model_name, plot, scheduled_lr_array):
    model = net.to(device)
    total_step = len(train_loader)
    overall_step = 0
    train_loss_values = []
    train_error = []
    val_loss_values = []
    val_error = []
    for epoch in range(epochs):
        correct = 0
        total = 0
        flag = 0
        running_loss = 0.0

        adjust_learning_rate(optimizer, epoch, scheduled_lr_array)
        print(f"Epoch {epoch+1}, Learning Rate: {optimizer.param_groups[0]['lr']}")

        for i, (images, labels) in enumerate(train_loader):
            # Move tensors to configured device
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            optimizer.step()
            if (i+1) % 1000 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))
            if plot:
              info = { ('loss_' + model_name): loss.item() }

              for tag, value in info.items():
                logger.add_scalar(tag, value, overall_step+1)
        train_loss_values.append(running_loss)
        train_error.append(100 - 100 * correct / total)

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for i, (images, labels) in enumerate(test_loader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))
        val_error.append(100-100*correct/total)
        val_loss_values.append(running_loss)
    return val_error,val_loss_values,train_error,train_loss_values


In [ ]:
model_path_100 = os.path.join(dir_root, 'runs/hw2p3_model_100_epoch.pt')

TRAIN_FLAG = True

if not os.path.exists(model_path_100) or TRAIN_FLAG:
    model = allcnn_t().to(device)
    epochs = 100
    scheduled_lr = [0.5, 0.007, 0.0005]
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.001, nesterov=True)

    val_error, val_loss_values, train_error, train_loss_values = train(model, optimizer, criterion, trainloader, testloader, epochs, 'cnn_curve_100', True, scheduled_lr)

    torch.save(model, model_path_100)

    np.save(os.path.join(dir_root, 'runs/train_error_100.npy'), np.array(train_error))
    np.save(os.path.join(dir_root, 'runs/train_loss_values_100.npy'), np.array(train_loss_values))
    np.save(os.path.join(dir_root, 'runs/val_error_100.npy'), np.array(val_error))
    np.save(os.path.join(dir_root, 'runs/val_loss_values_100.npy'), np.array(val_loss_values))
else:
    print(f"Model already exists at {model_path_100}, skipping training.")

Num parameters:  1667166
Epoch 1, Learning Rate: 0.5


In [ ]:
# Plot the training and validation losses and errors with the number of epochs
# First load the saved data
if os.path.exists(model_path_100) and not TRAIN_FLAG:
    train_error = np.load(os.path.join(dir_root, 'runs/train_error_100.npy'))
    train_loss_values = np.load(os.path.join(dir_root, 'runs/train_loss_values_100.npy'))
    val_error = np.load(os.path.join(dir_root, 'runs/val_error_100.npy'))
    val_loss_values = np.load(os.path.join(dir_root, 'runs/val_loss_values_100.npy'))

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(np.array(train_loss_values), label='Training Loss')
plt.plot(np.array(val_loss_values), label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(np.array(train_error), label='Training Error')
plt.plot(np.array(val_error), label='Validation Error')
plt.xlabel('Epoch')
plt.ylabel('Error (%)')
plt.title('Training and Validation Error')
plt.legend()
plt.show()


In [ ]:
# Automatically shutdown Colab
if 'google.colab' in str(get_ipython()):
    from google.colab import runtime
    runtime.unassign()
